# Monotonically decreasing SoH
The goal of this notebook is to demo a way of forcing our soh results to be monotonically decreasing.

## Setup

### Imports

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from core.stats_utils import *
from transform.results.config import *
from transform.results.main import get_results

### Data extraction

In [ ]:
results = get_results()

### Original resusls visualization
To understand the problem we are trying to solve, let's first visualize the original results.

In [ ]:
px.scatter(results, x="odometer", y="soh", color="model", title="Original SoH results")

### Demo

In [ ]:
# Application du traitement sur le DataFrame original
ts_processed = (results
            # 3. Appliquer le traitement de SoH par véhicule
            .groupby('vin')
            .apply(process_soh_for_vehicle)
            .reset_index(level=0, drop=True)  # Supprime l'index 'vin' créé par le groupby
            
            # 1. D'abord, arrondir les dates à la semaine
            .assign(date=lambda df: df["date"].dt.floor(UPDATE_FREQUENCY))
            
            
            # 2. Grouper par vin et date pour avoir une valeur hebdomadaire
            .groupby(["vin", "date"])
            .agg({
                "odometer": "last",    
                "soh": "median",
                "model": "first",
            })
            .pipe(filter_results_by_lines_bounds, VALID_SOH_POINTS)
            .reset_index()
            
            
            # 4. Reset final de l'index
            .reset_index(drop=True))

# Visualisation des résultats
fig = px.scatter(ts_processed, x="odometer", y="soh", color="model", 
                title="SoH optimisé en fonction du kilométrage")
fig.show()